In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)  # sets the max
import numpy as np
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")


from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score, f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

rs = 1  # random state seed for reproducibility

# Fair PCA Experiments



## Functions



### Fair PCA

In [2]:
def fair_PCA(X, protected_features, n_components):

    Z = np.copy(protected_features)

    # Removing the mean from the protected features
    Z = Z-np.mean(Z)

    # Finding the orthonormal null-space spanned by Z^t X
    R = scipy.linalg.null_space(np.matmul(Z.T, X))

    # Finding the orthonormal eigenvectors of R^T X^T X R
    vals, L = scipy.linalg.eig(np.linalg.multi_dot([np.transpose(R), np.transpose(X), X, R]))

    #sort by eigen values
    idx = vals.argsort()[::-1]
    L = L[:,idx]


    # Finding the projection matrix
    U = np.matmul(R, L[:n_components])

    # Projecting our data into fair space and returning X'
    return U, np.matmul(np.transpose(U),np.transpose(X))

### Fairness metrics (Equalized odds)

In [3]:
def fpr_and_tpr(cm):
    TN = cm[0][0]
    FN = cm[1][0]
    FP = cm[0][1]
    TP = cm[1][1]

    TPR = TP/(TP+FN)
    FPR = FP/(FP+TN)

    return FPR, TPR

def equalized_odds(
        model,
        X,
        y,
        groups,
        group_protected,
        group_non_protected
    ):
    
    X_protected = X[groups == group_protected]
    y_protected = y[groups == group_protected]
    predictions_protected = model.predict(X_protected)


    X_non_protected = X[groups == group_non_protected]
    y_non_protected = y[groups == group_non_protected]
    predictions_non_protected = model.predict(X_non_protected)

    cm_protected = confusion_matrix(y_protected, predictions_protected)
    cm_non_protected = confusion_matrix(y_non_protected, predictions_non_protected)

    FPR_protected, TPR_protected = fpr_and_tpr(cm_protected)
    FPR_non_protected, TPR_non_protected = fpr_and_tpr(cm_non_protected)

    return pd.DataFrame({"FPR":[FPR_protected, FPR_non_protected], "TPR":[TPR_protected,TPR_non_protected]}, index=[group_protected,group_non_protected])


### Main experiment class

In [4]:
class ExperimentBaseline:
    def __init__(
            self, 
            data,
            course,
            grade_threshold,
            test_ratio,
            random_state
        ):

        self.course = course
        self.random_state = random_state
        self.test_ratio = test_ratio

        self.data = data[data["course"] == self.course]

        self.target = data[data["course"] == self.course]["G3"].apply(lambda x: 0 if x < grade_threshold else 1)

        self.groups = data[data["course"] == self.course]["SES"]

        self.protected_variables = [
            "internet",
            "traveltime",
            "address",
            "Mjob",
            "Fjob",
            "Medu",
            "Fedu",
            "SES"
        ]

        self.groups_and_protected = data[data["course"] == self.course][self.protected_variables]


    def baseline_data_prep(self):

        one_hot_cols =[
            "school",
            "sex",
            "age",
            "address",
            "famsize",
            "Pstatus",
            "Mjob",
            "Fjob",
            "reason",
            "guardian"
        ]

        _data = pd.get_dummies(
                self.data,
                prefix=None,
                prefix_sep="_",
                dummy_na=False,  # dont add a column for missing values
                columns=one_hot_cols,  # the columns we create the dummies for
                drop_first=True,  # IMPORTANT to have true! removes the first dummy indicator. This is done to avoid multicollinearity. The category removed is indicated when all other dummy categories are 0.
            )

        _data = _data.replace({
            "schoolsup": {"no":False, "yes":True},
            "famsup": {"no":False, "yes":True},
            "paid": {"no":False, "yes":True},
            "activities": {"no":False, "yes":True},
            "nursery": {"no":False, "yes":True},
            "higher": {"no":False, "yes":True},
            "internet": {"no":False, "yes":True},
            "romantic": {"no":False, "yes":True},
        })

        _data = _data[_data["course"] == self.course][[
            'Medu', 'Fedu', 'traveltime', 'studytime', 'failures',
            'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher',
            'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc',
            'health', 'absences', 'school_MS', 'sex_M', 'age_16',
            'age_17', 'age_18', 'age_19', 'age_20', 'age_21', 'age_22', 'address_U',
            'famsize_LE3', 'Pstatus_T', 'Mjob_health', 'Mjob_other',
            'Mjob_services', 'Mjob_teacher', 'Fjob_health', 'Fjob_other',
            'Fjob_services', 'Fjob_teacher', 'reason_home', 'reason_other',
            'reason_reputation', 'guardian_mother', 'guardian_other'
            ]
        ]

        self.X_train, self.X_test, self.y_train, self.y_test, self.group_train, self.group_test = train_test_split(
            _data,
            self.target,
            self.groups,
            test_size=self.test_ratio,
            random_state=self.random_state
        )


    def no_protected_data_prep(self):

        one_hot_cols =[
            "school",
            "sex",
            "age",
            "famsize",
            "Pstatus",
            "reason",
            "guardian"
        ]

        _data = pd.get_dummies(
                self.data,
                prefix=None,
                prefix_sep="_",
                dummy_na=False,  # dont add a column for missing values
                columns=one_hot_cols,  # the columns we create the dummies for
                drop_first=True,  # IMPORTANT to have true! removes the first dummy indicator. This is done to avoid multicollinearity. The category removed is indicated when all other dummy categories are 0.
            )

        _data = _data.replace({
            "schoolsup": {"no":False, "yes":True},
            "famsup": {"no":False, "yes":True},
            "paid": {"no":False, "yes":True},
            "activities": {"no":False, "yes":True},
            "nursery": {"no":False, "yes":True},
            "higher": {"no":False, "yes":True},
            "romantic": {"no":False, "yes":True},
        })

        _data = _data[_data["course"] == self.course][[
            'studytime', 'failures','schoolsup', 'famsup', 'paid', 'activities', 
            'nursery', 'higher', 'romantic', 'famrel', 'freetime',
            'goout', 'Dalc', 'Walc', 'health', 'absences', 'school_MS', 'sex_M', 'age_16',
            'age_17', 'age_18', 'age_19', 'age_20', 'age_21', 'age_22', 
            'famsize_LE3', 'Pstatus_T', 'reason_home', 'reason_other',
            'reason_reputation', 'guardian_mother', 'guardian_other'
            ]
        ]

        self.X_train, self.X_test, self.y_train, self.y_test, self.group_train, self.group_test = train_test_split(
            _data,
            self.target,
            self.groups,
            test_size=self.test_ratio,
            random_state=self.random_state
        )


    def fair_pca_data_prep(self):
        one_hot_cols =[
            "school",
            "sex",
            "age",
            "famsize",
            "Pstatus",
            "reason",
            "guardian"
        ]

        _data = pd.get_dummies(
                self.data,
                prefix=None,
                prefix_sep="_",
                dummy_na=False,  # dont add a column for missing values
                columns=one_hot_cols,  # the columns we create the dummies for
                drop_first=True,  # IMPORTANT to have true! removes the first dummy indicator. This is done to avoid multicollinearity. The category removed is indicated when all other dummy categories are 0.
            )

        _data = _data.replace({
            "schoolsup": {"no":False, "yes":True},
            "famsup": {"no":False, "yes":True},
            "paid": {"no":False, "yes":True},
            "activities": {"no":False, "yes":True},
            "nursery": {"no":False, "yes":True},
            "higher": {"no":False, "yes":True},
            "romantic": {"no":False, "yes":True},
        })

        _data = _data[_data["course"] == self.course][[
            'studytime', 'failures','schoolsup', 'famsup', 'paid', 'activities', 
            'nursery', 'higher', 'romantic', 'famrel', 'freetime',
            'goout', 'Dalc', 'Walc', 'health', 'absences', 'school_MS', 'sex_M', 'age_16',
            'age_17', 'age_18', 'age_19', 'age_20', 'age_21', 'age_22', 
            'famsize_LE3', 'Pstatus_T', 'reason_home', 'reason_other',
            'reason_reputation', 'guardian_mother', 'guardian_other'
            ]
        ]

        self.X_train, self.X_test, self.y_train, self.y_test, self.group_train, self.group_test = train_test_split(
            _data,
            self.target,
            self.groups_and_protected,
            test_size=self.test_ratio,
            random_state=self.random_state
        )


    def train_and_test(
            self,
            model,
            model_parameters,
            eval
        ):

        _clf = model(random_state=self.random_state)
        self.clf = GridSearchCV(_clf, model_parameters, scoring=eval)
        self.clf.fit(self.X_train, self.y_train)

        
        self.predictions = self.clf.predict(self.X_test)

        self.performance_metrics = {
            "accuracy":accuracy_score(self.y_test, self.predictions),
            "recall":recall_score(self.y_test, self.predictions),
            "f1":f1_score(self.y_test, self.predictions)
        }

        self.fairness_metrics = equalized_odds(
            self.clf, 
            self.X_test,
            self.y_test,
            self.group_test,
            group_protected="lower",
            group_non_protected="middle/rich"
        )

        print(self.performance_metrics)
        print(self.fairness_metrics)

    def apply_fairpca(self):
        pass

    def train_and_test_pca(
            self,
            model,
            model_parameters,
            eval,
            protected_features_to_suppress
        ):
        
        _clf = model(random_state=self.random_state)
        self.clf = GridSearchCV(_clf, model_parameters, scoring=eval)





        self.clf.fit(self.X_train, self.y_train)

        
        self.predictions = self.clf.predict(self.X_test)

        self.performance_metrics = {
            "accuracy":accuracy_score(self.y_test, self.predictions),
            "recall":recall_score(self.y_test, self.predictions),
            "f1":f1_score(self.y_test, self.predictions)
        }

        self.fairness_metrics = equalized_odds(
            self.clf, 
            self.X_test,
            self.y_test,
            self.group_test,
            group_protected="lower",
            group_non_protected="middle/rich"
        )

        print(self.performance_metrics)
        print(self.fairness_metrics)


### Run all experiments function

## Data Prep

In [5]:
df = pd.read_csv("data/all_students_and_SES.csv")

features = df.columns
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,course,G_mean,SES_score,SES
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6,math,5.666667,-0.733077,middle/rich
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6,math,5.333333,0.176471,lower
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10,math,8.333333,0.176471,lower
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15,math,14.666667,-1.234671,middle/rich
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10,math,8.666667,-0.323278,middle/rich


In [6]:
rfc_parameters = {
    "n_estimators": [10, 20, 50, 75, 100, 1000],
    "max_depth": [5,10,15,20, None]
}
svc_parameters = {
    'kernel':["linear","poly","rbf"],
    'C':[0.001, 0.01, 0.1, 1, 10, 100],
    'gamma':["scale"]
}

## Baseline

### Grade threshold 10 (Passing grade)

#### Math

In [8]:
svm_baseline_math = ExperimentBaseline(
    data = df, 
    course = "math", 
    grade_threshold = 10, 
    test_ratio = 0.4, 
    random_state = rs
)

svm_baseline_math.baseline_data_prep()

svm_baseline_math.train_and_test(
    model = SVC, 
    model_parameters= {
        'kernel':["linear","poly","rbf"],
        'C':[0.001, 0.01, 0.1, 1, 10, 100],
        'gamma':["scale"]
    },
    eval="f1"
)


{'accuracy': 0.7215189873417721, 'recall': 0.9174311926605505, 'f1': 0.819672131147541}
                  FPR       TPR
lower        0.687500  0.896552
middle/rich  0.727273  0.925000


#### Portuguese

In [9]:
svm_baseline_portuguese = ExperimentBaseline(
    data = df, 
    course = "portuguese", 
    grade_threshold = 10, 
    test_ratio = 0.4, 
    random_state = rs
)

svm_baseline_portuguese.baseline_data_prep()

svm_baseline_portuguese.train_and_test(
    model = SVC, 
    model_parameters= {
        'kernel':["linear","poly","rbf"],
        'C':[0.001, 0.01, 0.1, 1, 10, 100],
        'gamma':["scale"]
    },
    eval="f1"
)


{'accuracy': 0.8384615384615385, 'recall': 0.9771689497716894, 'f1': 0.9106382978723404}
                  FPR       TPR
lower        0.933333  0.941176
middle/rich  0.818182  1.000000


### Grade threshold 13 (Eligible for higher education)

#### Math

In [10]:
svm_baseline_math = ExperimentBaseline(
    data = df, 
    course = "math", 
    grade_threshold = 13, 
    test_ratio = 0.4, 
    random_state = rs
)

svm_baseline_math.baseline_data_prep()

svm_baseline_math.train_and_test(
    model = SVC, 
    model_parameters= {
        'kernel':["linear","poly","rbf"],
        'C':[0.001, 0.01, 0.1, 1, 10, 100],
        'gamma':["scale"]
    },
    eval="f1"
)


{'accuracy': 0.6518987341772152, 'recall': 0.49019607843137253, 'f1': 0.47619047619047616}
                  FPR       TPR
lower        0.243243  0.375000
middle/rich  0.285714  0.511628


#### Portuguese

In [11]:
svm_baseline_portuguese = ExperimentBaseline(
    data = df, 
    course = "portuguese", 
    grade_threshold = 13, 
    test_ratio = 0.4, 
    random_state = rs
)

svm_baseline_portuguese.baseline_data_prep()

svm_baseline_portuguese.train_and_test(
    model = SVC, 
    model_parameters= {
        'kernel':["linear","poly","rbf"],
        'C':[0.001, 0.01, 0.1, 1, 10, 100],
        'gamma':["scale"]
    },
    eval="f1"
)


{'accuracy': 0.676923076923077, 'recall': 0.6083333333333333, 'f1': 0.6347826086956522}
                  FPR       TPR
lower        0.131579  0.435897
middle/rich  0.421875  0.691358


## Removing Protected features

In [12]:
svm_baseline_math = ExperimentBaseline(
    data = df, 
    course = "math", 
    grade_threshold = 10, 
    test_ratio = 0.4, 
    random_state = rs
)

svm_baseline_math.no_protected_data_prep()

svm_baseline_math.train_and_test(
    model = SVC, 
    model_parameters= {
        'kernel':["linear","poly","rbf"],
        'C':[0.001, 0.01, 0.1, 1, 10, 100],
        'gamma':["scale"]
    },
    eval="f1"
)


{'accuracy': 0.7215189873417721, 'recall': 0.8899082568807339, 'f1': 0.8151260504201681}
                  FPR       TPR
lower        0.625000  0.827586
middle/rich  0.666667  0.912500


## FairPCA